<a href="https://colab.research.google.com/github/AeroEng16/GNN_learning/blob/main/DeepmindDatasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Trying Deepmind Research Data**

In [2]:
!git clone https://github.com/google-deepmind/deepmind-research
!git clone https://github.com/Mohamedelrefaie/DrivAerNet

fatal: destination path 'deepmind-research' already exists and is not an empty directory.
Cloning into 'DrivAerNet'...
remote: Enumerating objects: 686, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 686 (delta 67), reused 51 (delta 51), pack-reused 605 (from 2)
Receiving objects: 100% (686/686), 1023.41 KiB | 14.83 MiB/s, done.
Resolving deltas: 100% (346/346), done.


In [6]:
!pip install DrivAerNet/requirements.txt

ERROR: Invalid requirement: 'DrivAerNet/requirements.txt': Expected end or semicolon (after name and no valid version specifier)
    DrivAerNet/requirements.txt
              ^
Hint: It looks like a path. The path does exist. The argument you provided (DrivAerNet/requirements.txt) appears to be a requirements file. If that is the case, use the '-r' flag to install the packages specified within it.


In [4]:
!python DrivAerNet/tutorials/DrivAerNet_tutorial.py

Figure(2000x1000)
Traceback (most recent call last):
  File "/content/DrivAerNet/tutorials/DrivAerNet_tutorial.py", line 79, in <module>
    import pyvista as pv
ModuleNotFoundError: No module named 'pyvista'


In [ ]:
!pip install -r deepmind-research/meshgraphnets/requirements.txt

ERROR: Could not find a version that satisfies the requirement tensorflow-gpu<2,>=1.15 (from versions: 2.12.0)
ERROR: No matching distribution found for tensorflow-gpu<2,>=1.15


In [ ]:
!mkdir -p ${DATA}
!bash deepmind-research/meshgraphnets/download_dataset.sh cylinder_flow ${DATA}

mkdir: missing operand
Try 'mkdir --help' for more information.
--2025-07-25 12:16:49--  https://storage.googleapis.com/dm-meshgraphnets/cylinder_flow/meta.json
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.207, 172.217.204.207, 172.217.203.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 883 [application/octet-stream]
Saving to: ‘/cylinder_flow/meta.json’

/cylinder_flow/meta 100%[===================>]     883  --.-KB/s    in 0s      

2025-07-25 12:16:50 (16.1 MB/s) - ‘/cylinder_flow/meta.json’ saved [883/883]

--2025-07-25 12:16:50--  https://storage.googleapis.com/dm-meshgraphnets/cylinder_flow/train.tfrecord
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.207, 172.217.204.207, 172.217.203.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.207|:443... connected.
HTTP request sent, awaiting

In [ ]:
!python deepmind.research